### 1º Voy a priorizar en primer lugar a los  diseñadores y desarrolladores (lugares con aglomeración de empresas de videojuegos y tecnológicas que hayan facturado más de 1M en su historia, las de videojuegos cumplen el perfil en este caso)

Me quedo de la base de datos Companies con las empresas de videojuegos que han facturado más de 1M, ejecuto el script que escribe el json con la consulta filtrando la base de datos todo con el formato adecuado

In [1]:
from getJsonGames1M import writeJsonOffice
writeJsonOffice()

Hago el mongoimport e importo la colección a la base de datos,indexo el campo position como índice geoespacial. Ahora puedo buscar los lugares donde se aglomeran estas empresas, y buscar la lista de ciudades. El scrip devuelve una lista de ciudades donde hay empresas que a 100km a la redonda tienen más de 5 empresas de videojuegos que han facturado más de 1M, creo que es suficiente para satisfacer a los desarrolladores y a los diseñadores. 

In [2]:
from getCitiesDevNDesigner import citiesDevNDesignerOK
citiesDevNDes = citiesDevNDesignerOK()

In [3]:
len(citiesDevNDes)

45

### 2º En las anteriores ciudades voy a buscar qué tal son las comunicaciones, así cumplo con el requisito de los ejecutivos, para ello ejecuto el script que buscará aeropuertos, estaciones de tren y autobús con la api de google place pasándole el diccionario de las ciudades con sus coordenadas. El criterio va a ser que al menos haya 1 aeropuerto y 10 estaciones de tren o metro en 20 km a la redonda. Es de suponer que todas las ciudades que cumplan el criterio también tendrán buenas comunicaciones por carretera.

In [4]:
from filterCitiesForExecutive import getCities

In [5]:
cities = getCities(citiesDevNDes)

In [6]:
len(cities)

37

### 3º Ahora voy a buscar ciudades con una buena aglomeración de Starbucks, pienso que 5 en 10 km a la redonda es adecuado

In [7]:
from filterCitiesStarbucks import filterStarbucks

In [8]:
cities = list(filter(filterStarbucks,cities))

In [9]:
len(cities)

37

### 4º El siguiente filtro que voy a aplicar a mi lista es uno que satisfaga a las familias, me ha parecido que el criterio de que haya más de 3 lugares con la palabra universidad en 10 km cuadrados es un buen criterio, si hay universidades también hay colegios y guarderías para los niños

In [14]:
from filterCitiesSchool import filterUniversity
cities = list(filter(filterUniversity,cities))


In [15]:
len(cities)

35

### 5º Ahora voy a buscar que haya una buenna aglomeración de bares en la ciudad, si hay bares habrá discotecas y demás establecimientos de ocio nocturno y diurno.55 bares en 10 km

In [16]:
from filterCitiesBar import filterBar
cities = list(filter(filterBar,cities))

In [17]:
len(cities)

34

### 6º Ciudades veganas, mas de 20 sitios que contengan 'vegan' en 10km a la redonda

In [21]:
from filterCitiesBar import filterVegan
cities = list(filter(filterVegan,cities))
len(cities)

29

### 7. Hago un scraping para  añadir a cada ciudad su índice de coste de vida, me quedaré con el más bajo, creo que eso es bueno tanto para los trabajadores como para la empresa, porque ese índice incluye la adquisición de bienes y servicios.

In [23]:
from getIndexCost import getLCostIndex
import pandas as pd
pd.DataFrame(getLCostIndex(cities)).sort_values(by ='liveCostIndex' ).reset_index().drop(columns = 'index').to_csv('../outputs/okByCost.csv')

In [24]:
pd.read_csv('../outputs/okByCost.csv').drop(columns = 'Unnamed: 0')

,city,lat,long,liveCostIndex
0,Cambridge,42.372200,-71.114382,69.03
1,Los Angeles,37.446823,-122.161523,78.99
2,Singapore,37.580304,-122.343679,80.23
3,Philadelphia,39.952576,-75.165252,83.49
4,Boston,42.349149,-71.062173,86.65
5,Brooklyn,40.714953,-73.968431,86.95
6,San Francisco,37.751236,-122.381152,89.36
7,New York,40.735782,-73.979035,100.00
8,Mountain View,37.399529,-122.067236,NaN
9,San Mateo,37.514257,-122.253732,NaN


#### Los NaN son sitios que no ha encontrado el scraping, es lógico pues se corresponden a distritos de ciudades que sí que se encontró en el scraping, es facilmente comprobable sólo echando un vistazo a los valores de latitud y longitud. Ya hay una ciudad que cumple todas las necesidades, es Cambridge

### 8. Obtengo todas la compañías de Cambridge con el formato adecuado para indexar

In [1]:
from getJsonCambridge1M import writeJsonOfficeCambridge
writeJsonOfficeCambridge()

Importo el Json como una nueva colección a mi base de datos, sobre esa colección elaboraré la solución final.


In [2]:
def officeCambridgeOk():
    from pymongo import MongoClient
    import pandas as pd
    from getMongoClient import getClient
    from getEnv import getVariable
    client = getClient()
    dbCompanies = client.companies
    cambridge1m = list(dbCompanies.cambridge.find())
    listCambridge = getListOffice2km(cambridge1m,dbCompanies)


    #listGame1M100K=list(filter(lambda df:True if df.name.count() in range(5,maximo+1) else False,listGame1M100K))

    return listCambridge


def getCompaniesNear(lat,long, min_Distance,db):
    return list(db.cambridge.find({
        "position": {
            "$near": {
               "$geometry": {
                  "type": "Point" ,
                  "coordinates": [ long , lat ]
               },
               "$minDistance": min_Distance
             }
       }
    }))

def getListOffice2km(listOffice,db):
    import pandas as pd
    listDf = []
    for ele in listOffice:
        df = pd.DataFrame(getCompaniesNear(ele['lat'],ele['long'],2000,db))
        listDf.append(df.reset_index().drop(columns=['index','_id','position']))
    return listDf

In [3]:
l = officeCambridgeOk()
len(l)

77

In [4]:
import pandas as pd
pd.DataFrame(l[0])

,name,category,city,money_raised,founded_year,lat,long
0,MetaCarta,mobile,Cambridge,$10M,2001,42.362360,-71.099692
1,CityVoter,web,Cambridge,$4.18M,2005,42.358184,-71.106389
2,galaxyadvisors,network_hosting,Cambridge,$600k,2007,42.393551,-71.128231
3,LocaModa,advertising,Cambridge,$7.9M,2007,42.358822,-71.104329
4,Quanterix,biotech,Cambridge,$41.5M,2007,42.366326,-71.091795
5,Upland Software,software,Cambridge,$23.3M,1998,42.365612,-71.092202
6,Greentech Media,news,Cambridge,$4.58M,2007,42.360475,-71.095416
7,WebLayers,software,Cambridge,$10M,2002,42.377535,-71.149057
8,Forrester Research,consulting,Cambridge,$0,1983,42.363200,-71.092378
9,Sirtris Pharmaceuticals,biotech,Cambridge,$82M,2004,42.363567,-71.091189
